Imports

In [14]:
import gurobipy as gb
import pandas as pd
import numpy as np
import ast


Column Generation from Pre-processed Data

In [22]:
def indices_to_list(indices, length):
    return [1 if i in indices else 0 for i in range(length)]

In [23]:
pre_data = pd.read_csv('combinations.tsv',sep='\t')
pre_data['Chairs'] = pre_data['Chairs'].apply(ast.literal_eval)
pre_data['Members'] = pre_data['Members'].apply(ast.literal_eval)
pre_data['Chairs'] = pre_data['Chairs'].apply(lambda x: indices_to_list(x,10))
pre_data['Members'] = pre_data['Members'].apply(lambda x: indices_to_list(x,30))
pre_data['a'] = pre_data['Members'] + pre_data['Chairs']
pre_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16344393 entries, 0 to 16344392
Data columns (total 5 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   Day      int64 
 1   Time     int64 
 2   Members  object
 3   Chairs   object
 4   a        object
dtypes: int64(2), object(3)
memory usage: 623.5+ MB


In [20]:
data = pre_data[['Day','Time','a']]

In [21]:
data['h'] = data.groupby(['Day', 'Time'])['a'].cumcount() + 1


/var/folders/tf/frrjsdzx0fj9r1dvbn0zqj5m0000gn/T/ipykernel_96512/107800561.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['h'] = data.groupby(['Day', 'Time'])['a'].cumcount() + 1


In [11]:
data

,Day,Time,a,h
0,1,1,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,1,1,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2
2,1,1,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3
3,1,1,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4
4,1,1,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5
...,...,...,...,...
16344388,5,34,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",149496
16344389,5,34,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",149497
16344390,5,34,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",149498
16344391,5,34,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",149499


The meat and potatoes of it all...

In [12]:
def maximize_meetings(data,filename='sol'):
    model = gb.Model("Meeting Scheduling")

    # Decision variables
    meetings = {}

    # Indices
    times = range(1,35)
    days = range(1,6)
    persons = range(0,40)

    #Generated columns hidden with dict{ day : dict{ time : [col_1,col_2,...col_h]}}
    cols = {
        1 : {}, #here would contain times 1-34, followed by a list of list/vectors/cols
        2 : {},
        3 : {},
        4 : {},
        5 : {}
    }
    for d in days: # days 1-5
        for t in times: # time slots 1-34
            cols[d][t] = data[(data['Day'] == d) & (data['Time'] == t)]['a'].to_numpy()#.to_list()
            for h in range(len(cols[d][t])): #iterate over possible columns
                # Decision variables
                meetings[(t, d, h)] = model.addVar(vtype=gb.GRB.BINARY, name=f"x_{t}_{d}_{h}")

    for d in days: # days 1-5
        for t in times: # time slots 1-34
            num_cols = len(cols[d][t])

            # Constraint 1: At most 4 meetings per time slot and day
            model.addConstr(gb.quicksum(meetings[(t, d, h)] for h in range(num_cols)) <= 4)

            # Constraint 2: Each participant attends at most one meeting per time slot and day
            for p in persons:
                model.addConstr(gb.quicksum(meetings[(t, d, h)] for h in range(num_cols)
                                      if cols[d][t][h][p] == 1) <= 1)
                
            # Constraint 3: No back-to-back meetings for participants
            if t != 34:
                for h in range(len(cols[d][t])):   
                    for h2 in range(len(cols[d][t+1])): # this is crucial as h varies!!!
                        for p in persons:
                            #print(t, d, h, p)
                            if cols[d][t][h][p] == 1 and cols[d][t+1][h2][p] == 1:
                                model.addConstr(meetings[(t, d, h)] + meetings[(t+1, d, h2)] <= 1)


    # Objective: Maximize number of meetings
    model.setObjective(gb.quicksum(meetings[(t, d, h)] for t in times
                                    for d in days for h in range(len(cols[d][t]))), gb.GRB.MAXIMIZE)

    print('solving model...')
    
    # Solve the optimization problem
    model.optimize()

    # Print solution
    if model.status == gb.GRB.OPTIMAL:
        with open(f'{filename}.tsv', 'w') as sol:
            sol.truncate(0)
            sol.write('D\tT\ta*\n')
            print("Optimal solution found!")
            for d in days:
                for t in times:
                    for h in range(len(cols[d][t])):
                        if meetings[(t, d, h)].x > 0.5:
                            sol.write(f"{d}\t{t}\t{cols[d][t][h]}\n")
    else:
        print("No solution found.")
    return model


In [41]:
m = maximize_meetings(data[data['h'] <= 10],'sol_lt_10')

solving model...
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.3.0 21D62)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 45384 rows, 1630 columns and 85030 nonzeros
Model fingerprint: 0x1838b639
Variable types: 0 continuous, 1630 integer (1630 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 85.0000000
Presolve removed 45384 rows and 1630 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.05 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 85 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.500000000000e+01, best bound 8.500000000000e+01, gap 0.0000%
Optimal solution found!


In [42]:
m = maximize_meetings(data[data['h'] <= 500],'sol_lt_500')

solving model...
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.3.0 21D62)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50427552 rows, 77809 columns and 101232100 nonzeros
Model fingerprint: 0x66d5867b
Variable types: 0 continuous, 77809 integer (77809 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 89.0000000
Presolve removed 0 rows and 0 columns (presolve time = 6s) ...
Presolve removed 1987 rows and 0 columns (presolve time = 13s) ...
Presolve removed 1987 rows and 0 columns (presolve time = 15s) ...
Presolve removed 1987 rows and 0 columns (presolve time = 22s) ...
Presolve removed 1987 rows and 0 columns (presolve time = 25s) ...
Presolve removed 1987 rows and 0 columns (presolve time = 31s) ...
Presolve removed 1987 ro

In [52]:

m = maximize_meetings(data[data['h'] % 200 == 0],'sol_mod_200')

solving model...
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.3.0 21D62)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 48709724 rows, 81644 columns and 97881489 nonzeros
Model fingerprint: 0x578ccefb
Variable types: 0 continuous, 81644 integer (81644 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 269.0000000
Presolve removed 0 rows and 0 columns (presolve time = 7s) ...
Presolve removed 2664 rows and 0 columns (presolve time = 10s) ...
Presolve removed 2664 rows and 0 columns (presolve time = 16s) ...
Presolve removed 2694 rows and 0 columns (presolve time = 23s) ...
Presolve removed 2694 rows and 0 columns (presolve time = 26s) ...
Presolve removed 2694 rows and 0 columns (presolve time = 32s) ...
Presolve removed 2694 ro

In [11]:
baby_data = pd.read_csv('combinations2.tsv',sep='\t')
baby_data['Chairs'] = baby_data['Chairs'].apply(ast.literal_eval)
baby_data['Members'] = baby_data['Members'].apply(ast.literal_eval)
baby_data['Chairs'] = baby_data['Chairs'].apply(lambda x: indices_to_list(x,10))
baby_data['Members'] = baby_data['Members'].apply(lambda x: indices_to_list(x,30))
baby_data['a'] = baby_data['Members'] + baby_data['Chairs']
baby_data

,Day,Time,Members,Chairs,a
0,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,2,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,2,2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,2,3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
baby_m = maximize_meetings(baby_data,'baby_sol') 

solving model...
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.3.0 21D62)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6974 rows, 7 columns and 27 nonzeros
Model fingerprint: 0x1a71d425
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 4.0000000
Presolve removed 6974 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4.000000000000e+00, gap 0.0000%
Optimal solution found!


In [25]:
def maximize_meetings_on_day(data,filename='sol',day=0):
    model = gb.Model("Meeting Scheduling")

    # Decision variables
    meetings = {}

    # Indices
    times = range(1,35)
    persons = range(0,40)

    #Generated columns hidden with dict{ day : dict{ time : [col_1,col_2,...col_h]}}
    cols = {} #here would contain times 1-34, followed by a list of list/vectors/cols
            
    for t in times: # time slots 1-34
        cols[t] = data[(data['Time'] == t)]['a'].to_numpy()#.to_list()
        for h in range(len(cols[t])): #iterate over possible columns
            # Decision variables
            meetings[(t, h)] = model.addVar(vtype=gb.GRB.BINARY, name=f"x_{t}_{h}")

    for t in times: # time slots 1-34
        num_cols = len(cols[t])

        # Constraint 1: At most 4 meetings per time slot and day
        model.addConstr(gb.quicksum(meetings[(t, h)] for h in range(num_cols)) <= 4)

        # Constraint 2: Each participant attends at most one meeting per time slot and day
        for p in persons:
            model.addConstr(gb.quicksum(meetings[(t, h)] for h in range(num_cols)
                                    if cols[t][h][p] == 1) <= 1)
            
        # Constraint 3: No back-to-back meetings for participants
        if t != 34:
            for h in range(len(cols[t])):   
                for h2 in range(len(cols[t+1])): # this is crucial as h varies!!!
                    for p in persons:
                        #print(t, h, p)
                        if cols[t][h][p] == 1 and cols[t+1][h2][p] == 1:
                            model.addConstr(meetings[(t, h)] + meetings[(t+1, h2)] <= 1)


    # Objective: Maximize number of meetings
    model.setObjective(gb.quicksum(meetings[(t, h)] for t in times
                                    for h in range(len(cols[t]))), gb.GRB.MAXIMIZE)

    print('solving model...')
    
    # Solve the optimization problem
    model.optimize()

    # Print solution
    if model.status == gb.GRB.OPTIMAL:
        with open(f'{filename}.tsv', 'w') as sol:
            sol.truncate(0)
            sol.write('D\tT\ta*\n')
            print("Optimal solution found!")
            for t in times:
                for h in range(len(cols[t])):
                    if meetings[(t, h)].x > 0.5:
                        sol.write(f"{day}\t{t}\t{cols[t][h]}\n")
    else:
        print("No solution found.")
    return model


In [27]:
m_d1 = maximize_meetings_on_day(data[data['Day'] == 1],'sol_day1',1)

KeyboardInterrupt: 